In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import get_test_data
from mpl_toolkits.mplot3d import Axes3D
from fenics import *
from mshr import *
import numpy as np
from dotenv import load_dotenv
from scipy.ndimage import zoom
from scipy.interpolate import interp1d
import os
set_log_level(30)
# Cargar las variables del archivo .env
load_dotenv()

In [ ]:
nueva_ruta = '/media/erick/TOSHIBA EXT/tests'

In [ ]:
# Define time parameters
T = float(os.getenv('T'))
dt = float(os.getenv('dt'))
nb = int(os.getenv('nb'))
save_images = os.getenv('SAVE_IMAGES')

# Define time parameters
T = float(os.getenv('T'))
dt = float(os.getenv('dt'))
nb = 1

os.chdir(nueva_ruta)


In [ ]:
result_dir = "correlation_results"
os.makedirs(result_dir, exist_ok=True)
timesteps = np.arange(0, T, dt)
fields = ['c', 's', 'i']

In [ ]:
def compute_correlation_anisotropic(field, max_distance, num_angles=36):
    """
    Calcula la función de correlación C(r, theta) para un campo no isotrópico.
    
    field: matriz 2D del campo.
    max_distance: distancia máxima para calcular la correlación.
    num_angles: número de ángulos (direcciones) para muestreo.
    """
    size_x, size_y = field.shape
    mean_field = np.mean(field)
    var_field = np.var(field)
    
    # Discretizar ángulos
    angles = np.linspace(0, 2 * np.pi, num_angles, endpoint=False)
    distances = np.arange(1, max_distance + 1)
    
    correlation = np.zeros((num_angles, len(distances)))
    
    for a, theta in enumerate(angles):
        dx = np.cos(theta)  # Desplazamiento en x
        dy = np.sin(theta)  # Desplazamiento en y
        for d, r in enumerate(distances):
            correlation_r = 0
            count = 0
            for i in range(size_x):
                for j in range(size_y):
                    x2 = int(i + r * dx)
                    y2 = int(j + r * dy)
                    if 0 <= x2 < size_x and 0 <= y2 < size_y:  # Dentro de los límites
                        correlation_r += (field[i, j] - mean_field) * (field[x2, y2] - mean_field)
                        count += 1
            if count > 0:
                correlation[a, d] = correlation_r / (count * var_field)
    
    return angles, distances, correlation

In [ ]:
def correlation_length_anisotropic(distances, correlation, threshold=1/np.e):
    """
    Calcula la longitud de correlación para cada dirección (ángulo).
    """
    lengths = []
    for corr in correlation:
        # Encontrar el punto donde la correlación cae a 1/e
        idx = np.where(corr < threshold)[0]
        if len(idx) > 0:
            lengths.append(distances[idx[0]])
        else:
            lengths.append(distances[-1])  # Si no decae, usar la distancia máxima
    return np.array(lengths)

In [ ]:

# Leer un campo de ejemplo
field = np.loadtxt("matrix_c_0.000_nb_1.txt", delimiter="\t")

# Calcular la función de correlación anisotrópica
max_distance = 30  # Ajusta según el tamaño del campo
angles, distances, correlation = compute_correlation_anisotropic(field, max_distance)

# Calcular la longitud de correlación para cada dirección
xi_anisotropic = correlation_length_anisotropic(distances, correlation)

# Graficar la longitud de correlación como función de los ángulos
plt.figure(figsize=(8, 8))
plt.polar(angles, xi_anisotropic, marker='o', label="Longitud de correlación $\\xi(\\theta)$")
plt.title("Longitud de correlación en un campo no isotrópico")
plt.legend()
plt.show()

# Opcional: Graficar la función de correlación para un ángulo específico
angle_idx = 0  # Selecciona un ángulo
plt.figure(figsize=(8, 5))
plt.plot(distances, correlation[angle_idx], label=f"C(r) para $\\theta={angles[angle_idx]:.2f}$ rad")
plt.axvline(x=xi_anisotropic[angle_idx], color='r', linestyle='--', label=f"$\\xi(\\theta)={xi_anisotropic[angle_idx]:.2f}$")
plt.xlabel("Distancia $r$")
plt.ylabel("Correlación $C(r)$")
plt.title("Función de correlación en una dirección específica")
plt.legend()
plt.grid()
plt.show()